In [1]:
import sys
import os

# Esto busca la ruta a la carpeta raíz del proyecto (LatentLens)
# y la añade al "mapa" de Python temporalmente.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [3]:
# --- Pregunta B: Modelo de Filtrado Colaborativo (Versión 2.0 - Anti-MemoryError) ---

import pandas as pd
from src.data_loader import load_and_prepare_data

# --- PASO 0: Cargar nuestros datos ---
main_df = load_and_prepare_data()

# --- NUEVO PASO: Reducir el tamaño del dataset ---
# Para evitar el MemoryError, vamos a crear un prototipo con un subconjunto de los datos.

# 1. Encontrar las películas más populares
movie_counts = main_df['title'].value_counts()
# Nos quedamos con las 5000 películas más votadas
popular_movies = movie_counts.head(5000).index

# 2. Encontrar los usuarios más activos
user_counts = main_df['userId'].value_counts()
# Nos quedamos con los 10000 usuarios más activos
active_users = user_counts.head(10000).index

# 3. Filtramos nuestro DataFrame principal para que SOLO contenga
# las películas populares Y los usuarios activos.
df_filtered = main_df[main_df['title'].isin(popular_movies) & main_df['userId'].isin(active_users)]

print(f"Tamaño original: {len(main_df)} filas.")
print(f"Tamaño reducido para el prototipo: {len(df_filtered)} filas.")


# --- AHORA, CONTINUAMOS CON EL PLAN ORIGINAL, PERO CON LOS DATOS FILTRADOS ---

# --- PASO 1: Crear el "Mapa de Gustos" (Matriz Usuario-Película) ---
# Usamos el DataFrame filtrado, que es mucho más pequeño.
movie_user_matrix = df_filtered.pivot_table(index='title', columns='userId', values='rating')
movie_user_matrix.fillna(0, inplace=True)


# --- PASO 2: Optimizar el Mapa ---
from scipy.sparse import csr_matrix
movie_user_matrix_sparse = csr_matrix(movie_user_matrix.values)


# --- PASO 3: Entrenar al Detective (NearestNeighbors) ---
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(movie_user_matrix_sparse)


# --- PASO 4: Pedir Recomendaciones ---
def get_recommendations(movie_title, model, data_matrix, movie_map):
    try:
        movie_index = list(movie_map.index).index(movie_title)
        distances, indices = model.kneighbors(data_matrix[movie_index], n_neighbors=6)
        
        print(f"Recomendaciones si te gustó '{movie_title}':")
        for i in range(1, len(distances.flatten())):
            recommended_movie = movie_map.index[indices.flatten()[i]]
            print(f"- {recommended_movie}")
    except ValueError:
        print(f"La película '{movie_title}' no está en nuestro dataset de prototipo.")


# --- ¡Probemos nuestro modelo! ---
get_recommendations('Toy Story (1995)', model_knn, movie_user_matrix_sparse, movie_user_matrix)
print("\n")
get_recommendations('Matrix, The (1999)', model_knn, movie_user_matrix_sparse, movie_user_matrix)

Cargando los datasets...
Datos de películas cargados: 62423 filas.
Datos de calificaciones cargados: 25000095 filas.
Preprocesamiento completado. DataFrame final listo.
Tamaño original: 25000095 filas.
Tamaño reducido para el prototipo: 7785420 filas.
Recomendaciones si te gustó 'Toy Story (1995)':
- Back to the Future (1985)
- Jurassic Park (1993)
- Toy Story 2 (1999)
- Matrix, The (1999)
- Forrest Gump (1994)


Recomendaciones si te gustó 'Matrix, The (1999)':
- Forrest Gump (1994)
- Pulp Fiction (1994)
- Star Wars: Episode IV - A New Hope (1977)
- Star Wars: Episode V - The Empire Strikes Back (1980)
- Jurassic Park (1993)


In [ ]:
# --- Pregunta B: Modelo de Filtrado Colaborativo (Versión 3.0 - Optimizada para 32GB RAM) ---

import pandas as pd
from src.data_loader import load_and_prepare_data

# --- PASO 0: Cargar nuestros datos ---
main_df = load_and_prepare_data()

# --- NUEVO PASO (VERSIÓN 3.0): Expandir el prototipo para 32GB de RAM ---
# Con más memoria, podemos permitirnos un subconjunto de datos mucho más grande.
# Esto resultará en un modelo con más "conocimiento" y recomendaciones más precisas.

# 1. Encontrar las películas más populares
movie_counts = main_df['title'].value_counts()
# Nos quedamos con las 20,000 películas más votadas (antes eran 5,000)
popular_movies = movie_counts.head(15000).index

# 2. Encontrar los usuarios más activos
user_counts = main_df['userId'].value_counts()
# Nos quedamos con los 40,000 usuarios más activos (antes eran 10,000)
active_users = user_counts.head(40000).index

# 3. Filtramos nuestro DataFrame principal
df_filtered = main_df[main_df['title'].isin(popular_movies) & main_df['userId'].isin(active_users)]

print(f"Tamaño original: {len(main_df)} filas.")
print(f"Tamaño reducido para el prototipo (32GB RAM): {len(df_filtered)} filas.")


# --- AHORA, CONTINUAMOS CON EL PLAN ORIGINAL ---

# --- PASO 1: Crear el "Mapa de Gustos" (Matriz Usuario-Película) ---
movie_user_matrix = df_filtered.pivot_table(index='title', columns='userId', values='rating')
movie_user_matrix.fillna(0, inplace=True)


# --- PASO 2: Optimizar el Mapa ---
from scipy.sparse import csr_matrix
movie_user_matrix_sparse = csr_matrix(movie_user_matrix.values)


# --- PASO 3: Entrenar al Detective (NearestNeighbors) ---
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(movie_user_matrix_sparse)


# --- PASO 4: Pedir Recomendaciones ---
def get_recommendations(movie_title, model, data_matrix, movie_map):
    try:
        movie_index = list(movie_map.index).index(movie_title)
        distances, indices = model.kneighbors(data_matrix[movie_index], n_neighbors=6)
        
        print(f"Recomendaciones si te gustó '{movie_title}':")
        for i in range(1, len(distances.flatten())):
            recommended_movie = movie_map.index[indices.flatten()[i]]
            print(f"- {recommended_movie}")
    except ValueError:
        print(f"La película '{movie_title}' no está en nuestro dataset de prototipo.")
        print("CONSEJO: Asegúrate de que el título y el año son exactos.")


# --- ¡Probemos nuestro modelo con películas de mafia! ---
# Consejo: Para encontrar el título exacto, puedes usar esto en una celda:
# main_df[main_df['title'].str.contains('Godfather', case=False)]

get_recommendations('Godfather, The (1972)', model_knn, movie_user_matrix_sparse, movie_user_matrix)
print("\n")
get_recommendations('Goodfellas (1990)', model_knn, movie_user_matrix_sparse, movie_user_matrix)

Cargando los datasets...
Datos de películas cargados: 62423 filas.
Datos de calificaciones cargados: 25000095 filas.
Preprocesamiento completado. DataFrame final listo.
Tamaño original: 25000095 filas.
Tamaño reducido para el prototipo (32GB RAM): 17100439 filas.
Recomendaciones si te gustó 'Godfather, The (1972)':
- Godfather: Part II, The (1974)
- Pulp Fiction (1994)
- Goodfellas (1990)
- Silence of the Lambs, The (1991)
- Shawshank Redemption, The (1994)


Recomendaciones si te gustó 'Goodfellas (1990)':
- Godfather, The (1972)
- Pulp Fiction (1994)
- Godfather: Part II, The (1974)
- Reservoir Dogs (1992)
- Fargo (1996)
